In [ ]:
# import package
import os
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
import math
import sys
import random
import csv
 
# mount colab on google drive
from google.colab import drive
drive.mount('/content/drive')
 
# change dir, root dir
os.chdir('/content')
 
# change dir, file dir
os.chdir('drive/My Drive/codes/socomp')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read GroupTopic.txt
gt_lines = []
with open('GroupTopic.txt') as gt_f:
    gt_lines = gt_f.readlines()
gt_f.close()

group_id = []
group_organizer = []
group_attribute = []
gt_count = 1
for gt_line in gt_lines:
    if gt_count % 2 == 1:
      gt_id, gt_organizer = gt_line.split()
      group_id.append(int(gt_id[1:]))
      if gt_organizer == 'null':
        group_organizer.append(int(-1))
      else:
        group_organizer.append(int(gt_organizer[1:]))
    else:
      read_one_group_attribute = gt_line.split()
      num_group_attribute_list = []
      for i in range(len(read_one_group_attribute)):
        num_group_attribute_list.append(int(read_one_group_attribute[i][1:]))
      group_attribute.append(num_group_attribute_list)
    gt_count += 1

gt_data = zip(group_id, group_organizer, group_attribute)
gt_df = pd.DataFrame(gt_data, columns = ['group_id', 'group_organizer', 'group_attribute'])
gt_df = gt_df[gt_df['group_organizer'] != -1]
gt_df = gt_df.drop('group_organizer', 1)
max_group_id = gt_df['group_id'].max()
group_amount = len(gt_df.index)
gt_df

,group_id,group_attribute
0,0,"[85, 89, 124, 234, 738, 2527, 3007, 3135, 5915..."
1,1,"[212, 512, 3554]"
2,2,[1566]
3,3,"[244, 309, 1322, 1502, 3336, 4422, 8649, 9816,..."
4,4,"[638, 9753, 9772, 15067, 15984, 18874, 19061]"
...,...,...
477,477,"[1429, 7929, 18340, 29702]"
478,478,"[484, 1307, 2064, 3880, 6364, 10117, 15236, 15..."
479,479,"[9772, 10002, 10024, 10413]"
480,480,"[1237, 1273, 1543, 10050, 10080, 10232, 17665,..."


In [ ]:
# read MemberTopic.txt
mt_lines = []
with open('MemberTopic.txt') as mt_f:
    mt_lines = mt_f.readlines()
mt_f.close()

member_id = []
member_attribute = []
mt_count = 1
for mt_line in mt_lines:
    if mt_count % 2 == 1:
      member_id.append(int(mt_line[1:-1]))
    else:
      read_one_member_attribute = mt_line.split()
      num_member_attribute_list = []
      for i in range(len(read_one_member_attribute)):
        num_member_attribute_list.append(int(read_one_member_attribute[i][1:]))
      member_attribute.append(num_member_attribute_list)
    mt_count += 1

mt_data = zip(member_id, member_attribute)
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df = mt_df[mt_df.astype(str)['member_attribute'] != '[]']
max_member_id = mt_df['member_id'].max()
member_amount = len(mt_df.index)
mt_df

,member_id,member_attribute
0,0,"[10485, 498]"
1,1,"[1201, 914, 933, 638, 590, 726, 242, 1724, 158..."
2,2,"[10495, 9753, 491, 1201, 638, 2926, 15892, 229..."
3,3,"[11660, 11046, 2151]"
4,4,"[1201, 933, 230, 2899, 1755, 3282, 1923, 228, ..."
...,...,...
73677,73677,[10050]
73680,73680,"[1155, 17195, 60, 17866, 10232, 17030, 713, 24..."
73681,73681,"[240, 3336, 495, 22440, 29431, 16551, 4422, 24..."
73682,73682,"[6761, 6490, 1526, 1307, 1330, 2088, 2064, 445..."


In [ ]:
# read Group-Event folder
group_event_id = []
event_id = []
event_limit = []
event_time = []
event_organizer = []
yes_id = []
no_id = []
maybe_id = []
root = "./GroupEvent"
for i in range(0, 482):
  file = "G" + str(i) + ".txt"
  full_path = os.path.join(root, file)
  ge_group_id = file[:-4]
  with open(full_path) as ge_f:
    ge_lines = ge_f.readlines()
  ge_f.close()
  ge_count = 0
  for ge_line in ge_lines:
    ge_count += 1
    if ge_count % 5 == 0:
      maybe_id.append(ge_line.split())
    elif ge_count % 4 == 0:
      read_one_no_id = ge_line.split()
      num_no_id_list = []
      for i in range(len(read_one_no_id)):
        if read_one_no_id[i] != 'null':
          num_no_id_list.append(int(read_one_no_id[i][1:]))
      no_id.append(num_no_id_list)
    elif ge_count % 3 == 0:
      read_one_yes_id = ge_line.split()
      num_yes_id_list = []
      for i in range(len(read_one_yes_id)):
        if read_one_yes_id[i] != 'null':
          num_yes_id_list.append(int(read_one_yes_id[i][1:]))
      yes_id.append(num_yes_id_list)
    elif ge_count % 2 == 0:
      event_organizer.append(ge_line.split())
    else:
      ge_event_id, ge_event_limit, ge_event_time = ge_line.split()
      event_id.append(int(ge_event_id[1:]))
      event_limit.append(ge_event_limit)
      event_time.append(ge_event_time)
      group_event_id.append(int(ge_group_id[1:]))
    if ge_count == 5:
      ge_count = 0

ge_data = zip(group_event_id, event_id, event_limit, event_time, event_organizer, yes_id, no_id, maybe_id)
ge_df = pd.DataFrame(ge_data, columns = ['group_id', 'event_id', 'event_limit', 'event_time', 'event_organizer', 'yes_id', 'no_id', 'maybe_id'])
ge_df[['event_limit', 'event_time']] = ge_df[['event_limit', 'event_time']].astype(int)

# remove column maybe_id
ge_df = ge_df.drop('maybe_id', 1)

# remove useless  columns 
ge_df = ge_df.drop('event_organizer', 1)
ge_df = ge_df.drop('event_limit', 1)
ge_df = ge_df.drop('no_id', 1)
ge_df = ge_df.drop('event_time', 1)
ge_df = ge_df[ge_df.astype(str)['yes_id'] != '[]']
max_event_id = ge_df['event_id'].max()
event_amount = len(ge_df.index) 
ge_df

,group_id,event_id,yes_id
0,0,932,[48462]
1,0,947,"[48462, 56494, 62049, 70873, 48571]"
2,0,1742,[48462]
3,0,3372,"[48462, 55059, 56494, 70873]"
4,0,3373,"[48462, 21224, 27288, 63893]"
...,...,...,...
55391,481,39040,"[24063, 30598, 53015, 54522, 55322, 55840, 565..."
55392,481,40188,"[24063, 30598, 34608, 41229, 45326, 46797, 553..."
55393,481,40672,"[30598, 38405, 45326, 52699, 55322, 56568, 622..."
55394,481,41507,"[30598, 55322, 55850]"


In [ ]:
# randomly shuffle the rows of ge_df
ge_df = ge_df.sample(frac=1)

# split ge_df into 5 parts
p1, p2, p3, p4, p5 = np.array_split(ge_df, 5)

# train_set_1, test_set_1
train_frames = [p2, p3, p4, p5]
train_set = pd.concat(train_frames)
test_set = p1
train_set

,group_id,event_id,yes_id
6946,83,9017,[32471]
6879,80,51848,"[69234, 69877, 2221, 6859, 13014]"
34783,309,34386,"[35768, 46969, 47200, 54460, 60828, 37, 505, 8..."
4568,57,53625,"[27243, 56784, 65759, 71845]"
41552,347,9251,[69515]
...,...,...,...
48874,419,44447,"[39280, 20069]"
32968,300,13835,"[41812, 64427, 1229, 6552, 7743, 7747, 8771]"
22184,235,27536,"[43643, 32215, 42037, 53053, 16299, 36527]"
31514,293,1980,"[45908, 41167, 65247]"


In [ ]:
# train_member_set
train_member_set = set()
train_amount = len(train_set.index)
for i in range(train_amount):
  yes_id = train_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  train_member_set.update(yes_id)
len(train_member_set)

53003

In [ ]:
# test_member_set
test_member_set = set()
test_amount = len(test_set.index)
for i in range(test_amount):
  yes_id = test_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  test_member_set.update(yes_id)
len(test_member_set)

26532

In [ ]:
# member_set
member_set = set()
member_amount = len(mt_df.index)
for i in range(member_amount):
  member_id = mt_df.iloc[i]['member_id']
  member_set.add(member_id)
len(member_set)

47289

In [ ]:
# common_member_set, train & test & mt_df
train_test_common_set = list(set(train_member_set).intersection(test_member_set))
common_member_set = list(set(train_test_common_set).intersection(member_set))
common_member_set = [int(i) for i in common_member_set]
len(common_member_set)

14371

In [ ]:
# update mt_df, 32s
member_id = common_member_set
member_attribute = [set() for i in range(len(member_id))]
for i in range(len(mt_df.index)):
  id = int(mt_df.iloc[i]['member_id'])
  if id in member_id:
    index =  member_id.index(id)
    attribute = set(mt_df.iloc[i]['member_attribute'])
    member_attribute[index] = attribute

mt_data = zip(member_id, member_attribute)
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df

,member_id,member_attribute
0,32768,{590}
1,1,"{914, 15892, 15514, 17700, 933, 1201, 1459, 16..."
2,32770,"{1793, 7044, 638, 17674, 31885, 31254, 20246, ..."
3,32773,{10117}
4,65541,"{579, 4392, 8652, 498, 2516, 2294, 638}"
...,...,...
14366,65525,"{105477, 68229, 264, 15760, 15892, 16660, 6374..."
14367,65526,{254}
14368,65529,"{257, 17030, 392, 15497, 264, 15117, 15892, 10..."
14369,65530,"{3009, 10117, 392, 491, 235, 498, 7319, 1502}"


In [ ]:
# group_attribute_set
group_amount = len(gt_df.index)
group_attribute_set = []
for i in range(group_amount):
  one_group_attribute = gt_df.iloc[i]['group_attribute']
  for j in range(len(one_group_attribute)):
    if one_group_attribute[j] not in group_attribute_set:
      group_attribute_set.append(one_group_attribute[j])
group_attribute_set = set(group_attribute_set)
print(len(group_attribute_set))

1132


In [ ]:
# member_attribute_set
member_amount = len(mt_df.index)
member_attribute_set = set()
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  member_attribute_set.update(one_member_attribute)
print(len(member_attribute_set))

10686


In [ ]:
# group&member attribute
gm_attribute = group_attribute_set.intersection(member_attribute_set)
len(gm_attribute)

1082

In [ ]:
# remove member_attribute (not in gm_attribute), and update mt_df
new_member_id = []
new_member_attribute = []
member_amount = len(mt_df.index)
for i in range(member_amount):
  new_member_id.append(mt_df.iloc[i]['member_id'])
  one_old_member_attribute = mt_df.iloc[i]['member_attribute']
  one_new_member_attribute = gm_attribute.intersection(one_old_member_attribute)
  new_member_attribute.append(one_new_member_attribute)

mt_data = zip(new_member_id, new_member_attribute)
mt_df = pd.DataFrame(mt_data, columns = ['member_id', 'member_attribute'])
mt_df

,member_id,member_attribute
0,32768,{590}
1,1,"{933, 230, 713, 1001, 590, 1998, 2767, 1201, 9..."
2,32770,"{1793, 17674, 20246, 31254, 19491, 933, 1322, ..."
3,32773,{10117}
4,65541,"{579, 4392, 8652, 498, 2516, 2294, 638}"
...,...,...
14366,65525,"{68229, 264, 16660, 15892, 7319, 933, 4392, 12..."
14367,65526,{254}
14368,65529,"{257, 17030, 392, 15497, 264, 15117, 15892, 10..."
14369,65530,"{3009, 10117, 392, 491, 235, 498, 7319, 1502}"


In [ ]:
member_topic_df = mt_df

In [ ]:
# mt_df safe check point

In [ ]:
# remove group_attribute (not in gm_attribute), and update gt_df
new_group_id = []
new_group_attribute = []
group_amount = len(gt_df.index)
for i in range(group_amount):
  new_group_id.append(gt_df.iloc[i]['group_id'])
  one_old_group_attribute = gt_df.iloc[i]['group_attribute']
  one_new_group_attribute = gm_attribute.intersection(one_old_group_attribute)
  new_group_attribute.append(one_new_group_attribute)

gt_data = zip(new_group_id, new_group_attribute)
gt_df = pd.DataFrame(gt_data, columns = ['group_id', 'group_attribute'])
gt_df

,group_id,group_attribute
0,0,"{3007, 738, 23842, 44612, 234, 19119, 16816, 1..."
1,1,"{512, 3554, 212}"
2,2,{1566}
3,3,"{19168, 10306, 4422, 3336, 8649, 1322, 10290, ..."
4,4,"{9772, 15984, 19061, 9753, 18874, 15067, 638}"
...,...,...
461,477,"{7929, 18340, 1429, 29702}"
462,478,"{484, 10117, 15236, 16165, 3880, 15720, 17028,..."
463,479,"{10024, 10002, 9772, 10413}"
464,480,"{10080, 17665, 10050, 17986, 1543, 32778, 3642..."


In [ ]:
# re-caculate member_attribute_set
member_amount = len(mt_df.index)
member_attribute_set = set()
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  member_attribute_set.update(one_member_attribute)
print(len(member_attribute_set))

1082


In [ ]:
# re-caculate group_attribute_set
group_amount = len(gt_df.index)
group_attribute_set = set()
for i in range(group_amount):
  one_group_attribute = gt_df.iloc[i]['group_attribute']
  group_attribute_set.update(one_group_attribute)
print(len(group_attribute_set))

1082


In [ ]:
# update train_set(rm member not in common_member_set)
common_member_set = set(common_member_set)
new_yes_id = []
train_set_amount = len(train_set.index)
# pirnt(train)
for i in range(train_set_amount):
  yes_id = train_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  yes_id = common_member_set.intersection(yes_id)
  new_yes_id.append(yes_id)
group_id = train_set['group_id']
event_id = train_set['event_id']
train_data = zip(group_id, event_id, new_yes_id)
train_set = pd.DataFrame(train_data, columns = ['group_id', 'event_id', 'yes_id'])
train_set

,group_id,event_id,yes_id
0,83,9017,{32471}
1,80,51848,"{69234, 69877, 13014}"
2,309,34386,"{47200, 37, 34408, 42410, 42795, 24090, 44539,..."
3,57,53625,{56784}
4,347,9251,{69515}
...,...,...,...
43243,419,44447,"{39280, 20069}"
43244,300,13835,"{7747, 41812, 7743}"
43245,235,27536,"{43643, 42037}"
43246,293,1980,{}


In [ ]:
# train_set safe check point

In [ ]:
# matrix UTS， 8min 4s
member_amount = len(mt_df.index)
column_amount = 1
np_mt = mt_df.to_numpy()
UTS = np.zeros((member_amount, member_amount), dtype=float)
for i in range(member_amount):
  i_attribute = np_mt[i][1]
  for j in range(column_amount, member_amount):
    j_attribute = np_mt[j][1]
    com_attribute = i_attribute.intersection(j_attribute)
    numer = float(len(com_attribute))
    denom = float(len(i_attribute) + len(j_attribute) - len(com_attribute))
    if denom != 0:
      UTS[i][j] = numer / denom
      UTS[j][i] = UTS[i][j]
  column_amount += 1

In [ ]:
# member, even_set, 3min 7s
train_amount = len(train_set.index)
member_amount = len(mt_df.index)
index = [*range(member_amount)]
member_id = mt_df['member_id']
dictionary = dict(zip(index, member_id))
np_train = train_set.to_numpy()
member_event_set = [set() for i in range(member_amount)]
for i in range(train_amount):
  member_set = np_train[i][2]
  for j in member_set:
    member_index = -1
    for key, value in dictionary.items():
      if j == value:
        member_index = key
        break
    member_event_set[member_index].add(np_train[i][1])

me_data = zip(member_id, member_event_set)
me_set = pd.DataFrame(me_data, columns = ['member_id', 'member_event_set'])
me_set

,member_id,member_event_set
0,32768,{24451}
1,1,"{3921, 30458, 8070}"
2,32770,"{3624, 2378, 6897, 35895, 5119}"
3,32773,"{24727, 24545, 24839}"
4,65541,"{54883, 9959, 19787, 14156, 13367}"
...,...,...
14366,65525,"{27296, 31169, 54049, 43302, 714, 54058, 42314..."
14367,65526,"{44320, 54908, 44321, 34889, 45751, 40046, 358..."
14368,65529,"{54761, 32731, 41844, 3561}"
14369,65530,"{7233, 66, 6897, 1427, 40148}"


In [ ]:
# matrix Sim, 11min 34s
member_amount = len(mt_df.index)
column_amount = 1
np_me = me_set.to_numpy() 
Sim = np.zeros((member_amount, member_amount), dtype=float)
for i in range(member_amount):
  i_event = np_me[i][1]
  for j in range(column_amount, member_amount):
    j_event = np_me[j][1]
    com_event = i_event.intersection(j_event)
    numer = float(len(com_event))
    denom = float(len(i_event) + len(j_event) - len(com_event))
    Sim[i][j] += UTS[i][j]
    if denom != 0:
      Sim[i][j] += (numer / denom)
    Sim[j][i] = Sim[i][j]
  column_amount += 1

In [ ]:
# Neighbor(a), 2min 38s, attention!!!! is a_index, not a_id
member_amount = len(mt_df.index)
index = [*range(member_amount)]
K = 10
neighbor = [set() for i in range(member_amount)]
for i in range(member_amount):
  index_Sim = zip(index, Sim[i])
  nonzero_index_Sim = [i for i in index_Sim if i[1] > 0]
  if len(nonzero_index_Sim) >= K:
    sorted_index_Sim = sorted(nonzero_index_Sim, key=lambda x: x[1], reverse=True)
    for j in range(K):
      neighbor[i].add(sorted_index_Sim[j][0])

In [ ]:
# update test_set(rm member not in common_member_set)
common_member_set = set(common_member_set)
new_yes_id = []
test_set_amount = len(test_set.index)
for i in range(test_set_amount):
  yes_id = test_set.iloc[i]['yes_id']
  yes_id = set(yes_id)
  yes_id = common_member_set.intersection(yes_id)
  new_yes_id.append(yes_id)
group_id = test_set['group_id']
event_id = test_set['event_id']
test_data = zip(group_id, event_id, new_yes_id)
test_set = pd.DataFrame(test_data, columns = ['group_id', 'event_id', 'yes_id'])
test_set

,group_id,event_id,yes_id
0,293,4106,{65318}
1,335,3516,"{52454, 15795, 22967, 10776, 42522}"
2,391,42510,"{64541, 38357, 66511}"
3,277,6270,{}
4,313,28691,"{73445, 34495}"
...,...,...,...
10808,275,10070,"{32856, 62922, 66618, 62015}"
10809,66,2515,{21374}
10810,170,37145,{63167}
10811,164,21656,"{30993, 23642}"


In [ ]:
# test_event, test_even_attibute
group_id = []
for i in range(len(gt_df.index)):
  group_id.append(gt_df.iloc[i]['group_id'])
test_amount = len(test_set.index)
group_attribute = gt_df['group_attribute']
event_attribute = [set() for i in range(test_amount)]
even_id = test_set['event_id']
for i in range(test_amount):
  one_group_id = test_set.iloc[i]['group_id']
  if one_group_id in group_id:
    group_index = group_id.index(one_group_id)
    one_event_attribute = group_attribute[group_index]
    event_attribute[i] = one_event_attribute

test_et_data = zip(even_id, event_attribute)
test_et_set = pd.DataFrame(test_et_data, columns = ['even_id', 'event_attribute'])
test_et_set

,even_id,event_attribute
0,4106,"{44580, 10213, 4422, 12261, 712, 16551, 17866,..."
1,3516,"{4422, 264, 15992, 15117, 17201, 1112, 1273}"
2,42510,{}
3,6270,"{1091, 4422, 17030, 264, 17866, 155051, 498, 1..."
4,28691,"{18724, 6696, 18988, 590, 15892, 2294, 15832, ..."
...,...,...
10808,10070,"{20423, 15117, 1998, 22608, 71412, 16181, 10232}"
10809,2515,{849}
10810,37145,"{16800, 43390, 17571, 4422, 15046, 17030, 1786..."
10811,21656,{1660}


In [ ]:
# matrix member-event, UE_test, 57s
member_amount = len(mt_df.index)
event_amount = len(test_set.index)
index = [*range(member_amount)]
dictionary = dict(zip(index, mt_df['member_id']))
UE_test = np.zeros((member_amount, event_amount), dtype=int)
for i in range(event_amount):
  member_id = test_set.iloc[i]['yes_id']
  for j in member_id:
    for key, value in dictionary.items():
      if j == value:
        UE_test[key][i] = 1
        break

In [ ]:
# matrix pred
threshold = 0.5
member_amount = len(mt_df.index)
event_amount = len(test_set.index)
pred = np.zeros((member_amount, event_amount), dtype=float)
for i in range(member_amount):
  i_attribute = mt_df.iloc[i]['member_attribute']
  print(i_attribute)
  i_neighbor = neighbor[i]
  sum_R_a = len(UE_test[i].nonzero())
  R_a = float(sum_R_a) / event_amount
  for j in range(event_amount):
    one_event_attribute = event_attribute[j]
    pred_numer = 0
    pred_denom = 0
    behind_plus = 0
    for n in i_neighbor:
      n_neighbor_attribute = mt_df.iloc[n]['member_attribute']
      n_neighbor_attribute.update(i_attribute)
      com_attribute = n_neighbor_attribute.intersection(one_event_attribute)
      if len(com_attribute) != 0:
        pred_numer += Sim[i][n]
      pred_denom += Sim[i][n]
      if pred_denom != 0:
        behind_plus = pred_numer / pred_denom
  pred[i][j] = R_a + behind_plus  
  if pred[i][j] > threshold:
    pred[i][j] = 1
  else:
    pred[i][j] = 0

In [ ]:
# matrix member-event, UE_test
UE_test = lil_matrix((max_member_id+1, max_event_id+1), dtype=np.int8)
for i in range(len(test_set_1)):
  UE_test_c = test_set_1.iloc[i]['event_id']
  UE_test_r_set = test_set_1.iloc[i]['yes_id']
  for j in range(len(UE_test_r_set)):
    UE_test_r = UE_test_r_set[j]
    UE_test[UE_test_r, UE_test_c] = 1
UE_test.toarray()

In [ ]:
# member_set and test_event_set
member_set = []
for i in range(member_amount):
  member_set.append(mt_df.iloc[i]['member_id'])
test_event_set = []
test_set_amount = len(test_set_1.index)
for i in range(test_set_amount):
  test_event_set.append(test_set_1.iloc[i]['event_id'])

In [ ]:
# evaluate result
UE_predict = pred
TP = 0
FP = 0
FN = 0
TN = 0
for i in range(member_amount):
  i_member_id = mt_df.iloc[i]['member_id']
  for j in range(test_set_amount):
    j_event_id = test_set_1.iloc[j]['event_id']
    if UE_predict[i, j] == 1:
      if UE_test[i, j] == 1:
        TP += 1
      else:
        FP += 1
    else:
      if UE_test[i, j] == 1:
        FN += 1
      else:
        TN += 1
Precision = 0
Recall = 0
Accuracy = 0
F1_score = 0
if TP+FP != 0:
  Precision = TP / (TP+FP)
if TP+FN != 0:
  Recall  =TP / (TP+FN)
if TP+TN+FP+FN != 0:
  Accuracy = (TP+TN) / (TP+TN+FP+FN)
if Precision+Recall != 0:
  F1_score = 2 * Precision * Recall / (Precision+Recall)